In [10]:
import json, re

In [5]:
with open("../data/dpo/test.jsonl") as f:
    content = '['+','.join(f.readlines())+']'
    dataset = json.loads(content)

In [7]:
# 转成{"prompt","chosen","rejected"}三元组, 并且给prompt加上模板和assistant:, 给其他两个字段加上<eos>

In [43]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/data/lixubin/models/Qwen/Qwen1.5-1.8B-Chat/')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [54]:
def process(example=dataset[7]):
    result = example['chosen'].split("\n\nAssistant: ")
    history = []
    for seg in result[:-1]:
        history += seg.split("\n\nHuman: ")
    history.pop(0)
    messages = []
    for index, line in enumerate(history):
        if index % 2 == 0:
            messages.append({"role":"user", "content":line})
        else:
            messages.append({"role":"assistant", "content":line})
    prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    chosen = result[-1]+tokenizer.eos_token
    rejected = example['rejected'].split("\n\nAssistant: ")[-1]+tokenizer.eos_token

    # print(prompt)
    # print(chosen)
    # print(rejected)
    # print("="*20)
    # print(example['chosen'])
    return {"prompt": prompt, "chosen": chosen, "rejected": rejected}

process()

{'prompt': "<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\nWhat are situations I should have an emergency fund for?<|im_end|>\n<|im_start|>assistant\nOoh, that’s a big question.  Let’s see if I can put together a good answer.  I know a few ideas.  Here’s one.  I know that people have emergency situations when they have to cover big unexpected expenses, or when they lose their job.  What do you think of that as a possible situation?<|im_end|>\n<|im_start|>user\nOkay, so losing job is one of them, yes<|im_end|>\n<|im_start|>assistant\nAnother one might be having a bad medical emergency.<|im_end|>\n<|im_start|>user\nOkay, yes. So far we have two ideas.<|im_end|>\n<|im_start|>assistant\nOne more thing I was thinking of is an unexpected home repair or emergency car repair.<|im_end|>\n<|im_start|>user\nOkay, those are good, too. So far we have four ideas. I can't think of any besides those four. How about you?<|im_end|>\n<|im_start|>assistant\nHow about emergenc

In [55]:
new_dataset = []
for example in dataset:
    new_dataset.append(process(example))

In [57]:
with open("/data/lixubin/MiraTrainer/src/data/dpo/new_test.json", 'w') as f:
    json.dump(new_dataset, f, indent=4, ensure_ascii=False)